## Identify the gender of the voice

7 metrics are: carat,cut,color,clarity,depth,table,size(x,y,z)

### Import csv data

In [1]:
import sys
sys.path.append('../common/')
import csv
import random

GENDER = {'male':0, 'female':1}
class voicegender_data:
    def __init__(self,filepath, batch_size=250, shuffle = True):
        self.batch_size = batch_size
        self.shuffle=shuffle
        self.data_list, self.label_list = self.load_data(self.read_csv(filepath))
        self.batch_index = []
        self.index = 0
        if self.shuffle:
            for i in range(len(self.label_list)):
                index = random.randint(0, len(self.label_list)-1)
                if not index in self.batch_index:
                    self.batch_index.append(index)
        else: 
            for i in range(len(self.label_list)):
                self.batch_index.append(i)
            print(len(self.batch_index))
        
    def read_csv(self,filename):
        data = []
        headline = []
        with open(filename, "r") as f:
            csv_reader = csv.reader(f)
            headline = next(csv_reader)
            for row in csv_reader:
                data.append(row)
                #print(row)            
        return [headline, data]

    def load_data(self, data, shuffle=True):
        gender_data = []
        gender_label = []
        for row in data[1]:
            gender_data.append([float(row[0]), float(row[1]),float(row[2]),float(row[3]),float(row[4]),float(row[5]),float(row[6]),
                                float(row[7]),float(row[8]),float(row[9]),float(row[10]),float(row[11]),float(row[12]),
                                float(row[13]),float(row[14]),float(row[15]),float(row[16]),float(row[17]),float(row[18]),
                                float(row[19])])
            gender_label.append(GENDER[row[20]])
        return gender_data,gender_label

    def get_batch_data(self):
        batch_data = []
        batch_label = []
        i = 0
        while i < self.batch_size:
            batch_data.append(self.data_list[self.batch_index[self.index]])
            batch_label.append(self.label_list[self.batch_index[self.index]])
            self.index += 1
            if self.index >=len(self.batch_index):
                self.index = 0
            i += 1
        return batch_data, batch_label

### Build the neural network

In [2]:
import sys
sys.path.append('../common/')
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
import numpy as np
from train_log import train_log

TRAIN_BATCH_SIZE = 1000
LEARN_RATE = 0.01


log_dir = './log_voicegender/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)    
log = train_log(log_dir)

ckpt_dir = './ckpt_voicegender/'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)     

train_data = voicegender_data('./voicegender_data/voicegender_train.csv', batch_size=TRAIN_BATCH_SIZE)
        
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 20])
    y_ = tf.placeholder(tf.int64, [None])
    
with tf.name_scope('hidden_layer_1'):
    W_1 = tf.Variable(tf.random_uniform([20,10]))
    b_1 = tf.Variable(tf.zeros([10]))
    node_1 = tf.nn.tanh(tf.matmul(x,W_1)+b_1)
    print(node_1.shape)

with tf.name_scope('output'):
    W_2 = tf.Variable(tf.random_uniform([10,2]))
    b_2 = tf.Variable(tf.zeros([2]))
    y = tf.nn.softmax(tf.matmul(node_1,W_2)+b_2)
    print(y.shape)

with tf.name_scope('loss'):
    loss =  tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=y_,name="cross_entropy_per_example"))

with tf.name_scope('train_step'):
    train_step = tf.train.GradientDescentOptimizer(LEARN_RATE).minimize(loss)    

with tf.name_scope('accuracy'):
    prediction =tf.argmax(y, 1)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction,y_), tf.float32))
    
init = tf.global_variables_initializer()  
saver = tf.train.Saver(max_to_keep=1)

(?, 10)
(?, 2)


### Training the model

In [ ]:
import time
with tf.Session() as sess:      
    sess.run(init)
    tf.get_default_graph().finalize()
    model_file=tf.train.latest_checkpoint(ckpt_dir)
    saver.restore(sess,model_file)
    for j in range(500000,1000000):
        #print(str(j)+ '-1 : '+time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
        input_data, train_label = train_data.get_batch_data()
    
        #print(str(j)+ '-2 : '+time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
        accuracy_val, loss_val, _ = sess.run([accuracy, loss, train_step], 
                         feed_dict={x: input_data, y_: train_label})
        #print(str(j)+ '-3 : '+time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
        if (j+1) % 10 == 0:
            print('.',end='')
        if (j+1) % 1000 == 0:
            print()
            #print(predict)
            print('Step %d: loss = %.2f， accuracy = %.2f'%(j,loss_val,accuracy_val))
    saver.save(sess,ckpt_dir+'simpl_nn_ckpt',global_step=j+1)
    print("Training is done.")

INFO:tensorflow:Restoring parameters from ./ckpt_voicegender/simpl_nn_ckpt-500000
....................................................................................................
Step 500999: loss = 0.38， accuracy = 0.94
....................................................................................................
Step 501999: loss = 0.38， accuracy = 0.93
....................................................................................................
Step 502999: loss = 0.38， accuracy = 0.93
....................................................................................................
Step 503999: loss = 0.38， accuracy = 0.93
....................................................................................................
Step 504999: loss = 0.39， accuracy = 0.92
....................................................................................................
Step 505999: loss = 0.39， accuracy = 0.93
............................................................

In [ ]:
import time

test_data = voicegender_data('./voicegender_data/voicegender_test.csv', batch_size=1000)

with tf.Session() as sess:      
    sess.run(init)
    tf.get_default_graph().finalize()
    model_file=tf.train.latest_checkpoint(ckpt_dir)
    saver.restore(sess,model_file)
    accuracy_val, loss_val = sess.run([accuracy, loss], 
                         feed_dict={x: test_data.data_list, y_: test_data.label_list})
    print('Test result: loss = %.2f， accuracy = %.2f'%(loss_val,accuracy_val))